In [3]:
# This script explores CNN model performance based on training image resolution.
# We test the following training image sizes:
# 64 x 64, 128 x 128, 224 x 224, 384 x 384

In [148]:
import numpy as np
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import pydot
import random

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback

        
from sklearn.metrics import recall_score, classification_report
from sklearn.datasets import make_classification

import json

In [26]:
images_size64_exp5_Pr_Po_Im = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_Po_Im.npy', allow_pickle=True)
images_size64_exp5_Pr_Im = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_Im.npy', allow_pickle=True)
images_size64_exp5_PrPo_Im = np.load('../../data/tidy/preprocessed_images/size64_exp5_PrPo_Im.npy', allow_pickle=True)
images_size64_exp5_Pr_PoIm = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_PoIm.npy', allow_pickle=True)

images_size128_exp5_Pr_Po_Im = np.load('../../data/tidy/preprocessed_images/size128_exp5_Pr_Po_Im.npy', allow_pickle=True)
images_size128_exp5_Pr_Im = np.load('../../data/tidy/preprocessed_images/size128_exp5_Pr_Im.npy', allow_pickle=True)
images_size128_exp5_PrPo_Im = np.load('../../data/tidy/preprocessed_images/size128_exp5_PrPo_Im.npy', allow_pickle=True)
images_size128_exp5_Pr_PoIm = np.load('../../data/tidy/preprocessed_images/size128_exp5_Pr_PoIm.npy', allow_pickle=True)

images_size224_exp5_Pr_Po_Im = np.load('../../data/tidy/preprocessed_images/size224_exp5_Pr_Po_Im.npy', allow_pickle=True)
images_size224_exp5_Pr_Im = np.load('../../data/tidy/preprocessed_images/size224_exp5_Pr_Im.npy', allow_pickle=True)
images_size224_exp5_PrPo_Im = np.load('../../data/tidy/preprocessed_images/size224_exp5_PrPo_Im.npy', allow_pickle=True)
images_size224_exp5_Pr_PoIm = np.load('../../data/tidy/preprocessed_images/size224_exp5_Pr_PoIm.npy', allow_pickle=True)

images_size384_exp5_Pr_Po_Im = np.load('../../data/tidy/preprocessed_images/size384_exp5_Pr_Po_Im.npy', allow_pickle=True)
images_size384_exp5_Pr_Im = np.load('../../data/tidy/preprocessed_images/size384_exp5_Pr_Im.npy', allow_pickle=True)
images_size384_exp5_PrPo_Im = np.load('../../data/tidy/preprocessed_images/size384_exp5_PrPo_Im.npy', allow_pickle=True)
images_size384_exp5_Pr_PoIm = np.load('../../data/tidy/preprocessed_images/size384_exp5_Pr_PoIm.npy', allow_pickle=True)

In [27]:
NUM_CLASS = 2
NUM_CHANNEL = 1

In [125]:
def splitData(image_array, prop = 0.80, seed_num = 111):
    """Returns training and test arrays of images with specified proportion - prop:1-prop"""
    random.Random(seed_num).shuffle(image_array)
    train_size = int(prop*np.shape(image_array)[0])
    train = image_array[:train_size]
    test = image_array[train_size:]
    return(train, test)

def getImageShape(image_array):
    """Returns shape of image from array of images, e.g. WIDTH x HEIGHT x NUM_CHANNELS"""
    if NUM_CHANNEL==1:
        image_shape = np.array([np.expand_dims(x[0],axis=2) for x in image_array]).shape[1:4]
    elif NUM_CHANNEL==3:
        image_shape = np.array([x[0] for x in image_array]).shape[1:4][::-1]
    print(image_shape)
    return image_shape

def getImageAndLabelArrays(image_label_tuple_array, num_channels = 1):
    """Separates array of tuples (image matrix, vector label) into array of image matrices and array of image labels"""
    if num_channels == 1:
        image_array = np.array([np.expand_dims(x[0],axis=2) for x in image_label_tuple_array])
    elif num_channels == 3:
        image_array = np.array([x[0] for x in image_label_tuple_array]) 
        image_array = np.moveaxis(image_array, 1, -1)
    label_array = np.array([x[1] for x in image_label_tuple_array])
    return(image_array, label_array)

#https://github.com/keras-team/keras/issues/5400#issuecomment-408743570
def check_units(y_true, y_pred):
    if y_pred.shape[1] != 1:
      y_pred = y_pred[:,1:2]
      y_true = y_true[:,1:2]
    return y_true, y_pred

def precision(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

   
def reset_weights(model):
    """This function re-initializes model weights at each compile"""
    for layer in model.layers: 
        if isinstance(layer, tf.keras.Model):
            reset_weights(layer)
            continue
    for k, initializer in layer.__dict__.items():
        if "initializer" not in k:
            continue
        # find the corresponding variable
        var = getattr(layer, k.replace("_initializer", ""))
        var.assign(initializer(var.shape, var.dtype))

# define callback to save batch-wise loss/accuracy; Source: https://stackoverflow.com/a/52206330/3023033
class Histories(Callback):
    def on_train_begin(self,logs={}):
        self.losses = []
        self.accuracies = []
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracies.append(logs.get('accuracy'))

# Callback to find metrics at epoch end (not perfectly implemented; TODO) # Source: https://stackoverflow.com/a/56485026/3023033
class Metrics(Callback):
    def __init__(self, x, y):
        self.x = x
        self.y = y if (y.ndim == 1 or y.shape[1] == 1) else np.argmax(y, axis=1)
        self.reports = []

    def on_epoch_end(self, epoch, logs={}):
        y_hat = np.asarray(self.model.predict(self.x))
        y_hat = np.where(y_hat > 0.5, 1, 0) if (y_hat.ndim == 1 or y_hat.shape[1] == 1)  else np.argmax(y_hat, axis=1)
        report = classification_report(self.y,y_hat,output_dict=True)
        self.reports.append(report)
        return
   
    # Utility method
    def get(self, metrics, of_class):
        return [report[str(of_class)][metrics] for report in self.reports]


In [41]:
input_image_shape_64 = getImageShape(images_size64_exp5_Pr_Im)
input_image_shape_128 = getImageShape(images_size128_exp5_Pr_Im)
input_image_shape_224 = getImageShape(images_size224_exp5_Pr_Im)
input_image_shape_384 = getImageShape(images_size384_exp5_Pr_Im)

(64, 64, 1)
(128, 128, 1)
(224, 224, 1)
(384, 384, 1)


In [65]:
# Can either load a previously saved model or define here.
# model = models.load_model('../../results/models/MODEL_NAME')
base_model_64 = models.Sequential([
    layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_64),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5), # randomly drop out 50% of the neuorns at each training step
    layers.Dense(64, activation="relu"), # flatten all outputs
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASS, activation="softmax")
])

base_model_128 = models.Sequential([
    layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_128),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5), # randomly drop out 50% of the neuorns at each training step
    layers.Dense(64, activation="relu"), # flatten all outputs
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASS, activation="softmax")
])

base_model_224 = models.Sequential([
    layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_224),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5), # randomly drop out 50% of the neuorns at each training step
    layers.Dense(64, activation="relu"), # flatten all outputs
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASS, activation="softmax")
])
base_model_384 = models.Sequential([
    layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_384),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5), # randomly drop out 50% of the neuorns at each training step
    layers.Dense(64, activation="relu"), # flatten all outputs
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASS, activation="softmax")
])
# Compile the model.
#opt = SGD(lr = 0.001) #default learning rate (lr) = 0.1
# opt = keras.optimizers.Adam(learning_rate=0.01)
#model.compile(loss='categorical_crossentropy',  optimizer = "adam", metrics=[precision,recall, f1, 'accuracy'])

In [139]:
# The resolution test will be conducted scenario by scenario.
def test_resolution_performance(scenario, model_list, num_epochs = 10, trial_seed = 1): 
    resolution_keys = [64, 128, 224, 384]
    performance_dict = dict.fromkeys(resolution_keys)
    if scenario == "Pr_Po_Im":
        imageset = [images_size64_exp5_Pr_Po_Im, images_size128_exp5_Pr_Po_Im, images_size224_exp5_Pr_Po_Im, images_size384_exp5_Pr_Po_Im]
    elif scenario == "Pr_Im":
        imageset = [images_size64_exp5_Pr_Im]#, images_size128_exp5_Pr_Im, images_size224_exp5_Pr_Im, images_size384_exp5_Pr_Im]
    elif scenario == "PrPo_Im":
        imageset = [images_size64_exp5_PrPo_Im, images_size128_exp5_PrPo_Im, images_size224_exp5_PrPo_Im, images_size384_exp5_PrPo_Im]
    elif scenario == "Pr_PoIm":
        imageset = [images_size64_exp5_Pr_PoIm, images_size128_exp5_Pr_PoIm, images_size224_exp5_Pr_PoIm, images_size384_exp5_Pr_PoIm]

    k = 0
    for resolution_set in imageset:
        res_key = resolution_keys[k]
        performance_dict[res_key] = {}
        
        training_images_and_labels, test_images_and_labels = splitData(resolution_set, prop = 0.80, seed_num = trial_seed)
        training_images, training_labels = getImageAndLabelArrays(training_images_and_labels)
        validation_images, validation_labels = getImageAndLabelArrays(test_images_and_labels)
        batch_training_histories = Histories()
        # metrics_multiclass = Metrics(validation_images,validation_labels)  TODO
        # K.clear_session()
        model = model_list[k] # select k'th model from list
        reset_weights(model) # re-initialize model weights
        model.compile(loss='categorical_crossentropy',  optimizer = "adam", metrics = ['accuracy'])
        hist = model.fit(training_images, training_labels, batch_size = 32, epochs = num_epochs, verbose=1, validation_data=(validation_images, validation_labels)) #, callbacks=[batch_training_histories])
        performance_dict[res_key]['scenario'] = scenario
        performance_dict[res_key]['image_size'] = res_key
        performance_dict[res_key]['metrics'] = hist
        k += 1    
    return(performance_dict)

In [ ]:
RESOLUTION_PERFORMANCE_METRICS_DIR = '../../results/resolution-tests'
def main(num_trials = 5):
    scenario_list = ["Pr_Po_Im", "Pr_Im", "PrPo_Im", "Pr_PoIm"]
    optimized_models = [base_model_64, base_model_128, base_model_224, base_model_384] # TO BE UPDATED
    scenario_performance_dict = dict.fromkeys(scenario_list)
    if not os.path.exists(RESOLUTION_PERFORMANCE_METRICS_DIR): # check if 'tidy/preprocessed_images' subdirectory does not exist
        os.makedirs(RESOLUTION_PERFORMANCE_METRICS_DIR) # if not, create it    
    for i in range(num_trials):
        for j in scenario_list:
            scenario_performance_dict[j] = test_resolution_performance(j, optimized_models, num_epochs = 10, trial_seed = 1 + i) #ultimately should be averaged across trials       
        scenario_filename = "resolution_performance_" + j + "_trial_" + str(i) + ".txt"
        with open(os.path.join(RESOLUTION_PERFORMANCE_METRICS_DIR, data_filename), 'w') as file:
            file.write(json.dumps(scenario_performance_dict)) # use `json.loads` to do the reverse
    return()

In [ ]:
# Plotting 
# p1[64]['metrics'].history['loss']
# plt.close('all')
# for m in ['recall', 'precision', 'f1-score']:
#     for c in [0,1,2]:
#         plt.plot(metrics_multiclass.get(m,c), label='Class {0} {1}'.format(c,m))
        
# plt.legend(loc='lower right')
# plt.show()

In [83]:
def plot_model_accuracy(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("Model Accuracy")
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Validation"], loc="lower right")
    plt.show()

def plot_model_batch_accuracy(hist):
    #plt.plot(hist.accuracies)
    plt.plot(hist.val_accuracies)
    plt.title("Model Accuracy")
    plt.ylabel("Accuracy")
    plt.xlabel("Iteration")
    #plt.legend(["Train", "Validation"], loc="lower right")
    plt.show()
    
def plot_model_loss(hist):
    plt.plot(hist.history["loss"])
    plt.plot(hist.history["val_loss"])
    plt.title("Model Loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Validation"], loc="upper right")
    plt.show()